In [46]:
import psycopg2
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

In [48]:
cd oakland-crime-housing/

/Users/danaezoule/Documents/oakland-crime-housing


In [25]:
execfile("database.py")

Creating Database
Connecting to Database
Creating Crime Table
Loading Shapes
Creating Crime Geom Table
Creating Feature Table


In [86]:
conn = psycopg2.connect("dbname=oakland user=danaezoule")
cur = conn.cursor()
cur.execute("SELECT * FROM area_features;")
df = pd.DataFrame(cur.fetchall())

In [87]:
cdf = df.copy()

In [44]:
'''
cdf.columns = ['Idx', 'OPD_RD', 'Date', 'Time', 'Lat', 'Lng', 'year', 'year_month', 'quality', 'nonviolent', 'car_break_in', 'car_theft', 'violent', 'geom', 'block_group']
cdf['quarter'] = pd.DatetimeIndex(cdf.Date).quarter
cdf['day_of_week'] = pd.DatetimeIndex(cdf.Date).dayofweek
cdf['week'] = pd.DatetimeIndex(cdf.Date).week
cdf['month'] = pd.DatetimeIndex(cdf.Date).month
cdf['day'] = pd.DatetimeIndex(cdf.Date).day
cdf['day_of_year'] = pd.DatetimeIndex(cdf.Date).dayofyear
cdf['week_of_year'] = pd.DatetimeIndex(cdf.Date).weekofyear
cdf['hour'] = [i.hour for i in cdf.Time]
#combine date and time column so I can plot over 'em
cdf['Datetime'] = pd.to_datetime(cdf['Date'].astype(str) + ' ' + cdf['Time'].astype(str))
'''

In [52]:
cdf.columns = ['Group_Block', 'Quality', 'Nonviolent', 'Vehicle_Break_In', 'Vehicle_Theft', 'Violent']

In [56]:
from sklearn.cluster import KMeans
km = KMeans()

In [36]:
clus = km.fit_predict(cdf[['Quality', 'Nonviolent', 'Vehicle_Break_In', 'Vehicle_Theft', 'Violent']])

In [37]:
clus

array([3, 3, 6, 3, 5, 3, 6, 3, 6, 3, 1, 3, 7, 3, 3, 6, 3, 6, 6, 3, 3, 1, 1,
       4, 6, 3, 5, 1, 1, 4, 6, 6, 1, 6, 6, 6, 3, 1, 6, 3, 3, 1, 6, 3, 6, 5,
       6, 6, 6, 3, 3, 1, 6, 3, 1, 0, 6, 1, 1, 1, 0, 1, 1, 3, 6, 6, 1, 1, 6,
       3, 6, 3, 5, 6, 3, 3, 1, 1, 6, 6, 3, 1, 5, 3, 1, 3, 6, 3, 3, 3, 5, 3,
       6, 1, 3, 6, 6, 6, 3, 3, 3, 2, 3, 1, 1, 3, 3, 3, 6, 3, 3, 3, 6, 6, 3,
       3, 5, 3, 3, 1, 1, 1, 6, 6, 3, 3, 1, 6, 6, 3, 1, 3, 3, 3, 6, 3, 3, 3,
       3, 3, 6, 1, 1, 1, 3, 6, 3, 3, 1, 5, 3, 3, 0, 3, 6, 1, 3, 6, 6, 1, 3,
       6, 6, 6, 6, 3, 6, 3, 3, 5, 6, 3, 5, 6, 3, 1, 2, 1, 5, 3, 1, 1, 0, 6,
       3, 1, 1, 6, 6, 1, 6, 1, 1, 6, 6, 1, 1, 3, 3, 3, 6, 1, 3, 3, 1, 3, 1,
       1, 6, 1, 3, 1, 1, 3, 1, 1, 6, 1, 1, 6, 3, 0, 1, 3, 3, 3, 5, 5, 3, 1,
       4, 3, 3, 5, 1, 6, 1, 3, 3, 1, 6, 3, 1, 3, 0, 6, 4, 1, 6, 3, 3, 3, 6,
       1, 5, 3, 6, 3, 3, 0, 0, 1, 3, 3, 3, 1, 6, 3, 3, 1, 1, 3, 1, 1, 3, 3,
       1, 6, 0, 4, 3, 6, 0, 3, 3, 6, 6, 3, 3, 7, 1, 3, 1, 3, 0, 0, 3, 3, 5,
       5, 6,

In [ ]:
'''Feature Engineering Brainstorming:
Normalization:
    Normalize by population: none, assume census divisions cover this
    Normalize by geography (square footage or meterage)
    Normalize by total crime count

Geographical:
    Census tracts, group blocks, or blocks

Time Group By:
    Month, quarter, year

Time features:
    Count for weekday or weekend
    Count for time of day (morning, afternoon, eve, night)
        Split by data. First hypothesis:
        Morning: 6am-noon
        Afternoon: noon-6pm
        Eve: 6pm-midnight
        Early: midnight-6am
    
Housing etc:
    Are Trulia neighborhoods census tracts? Can I get block group info from Trulia?
    Will the ACS be helpful? Can I get yearly or quarterly ACS information?
    
Time component:
    Create centroids from earliest data, map all points to same centroids
    Create new centroids for each year (with varied data) as below 
    
Data that varies from year to year:
    If I use it, can I detect similar centroids between years?
    Should I instead ignore this completely, despite losing Lovely connection?
'''

In [88]:
# Splitting data by year, clustering by first year, predicting clusters for following years.
cdf.columns = ['Group_Block', 'Year','Quality', 'Nonviolent', 'Vehicle_Break_In', 'Vehicle_Theft', 'Violent']

In [97]:
cdf.sort('Group_Block')

,Group_Block,Year,Quality,Nonviolent,Vehicle_Break_In,Vehicle_Theft,Violent
290,1723,2014,122,57,154,39,50
1876,1723,2012,72,77,116,53,87
236,1723,2010,125,91,23,57,73
208,1723,2013,50,98,119,93,88
1245,1723,2009,111,78,41,45,71
1130,1723,2011,90,111,54,50,82
272,1769,2009,32,61,6,6,27
1178,1769,2013,27,54,180,12,37
1262,1769,2010,23,69,8,13,47
85,1769,2011,19,68,23,8,29


In [90]:
from sklearn.cluster import KMeans
km = KMeans()
columns = ['Quality', 'Nonviolent', 'Vehicle_Break_In', 'Vehicle_Theft', 'Violent']

In [101]:
clus = km.fit_predict(cdf[cdf.Year == 2009].sort('Group_Block')[columns])

In [103]:
clus10 = km.predict(cdf[cdf.Year == 2010].sort('Group_Block')[columns])

In [78]:
clus11 = km.predict(cdf[cdf.Year == 2011][columns])

In [92]:
clus.shape

(341,)

In [94]:
zip(np.arange(0, len(clus)), clus)

[(0, 7),
 (1, 7),
 (2, 6),
 (3, 0),
 (4, 2),
 (5, 7),
 (6, 0),
 (7, 1),
 (8, 2),
 (9, 0),
 (10, 0),
 (11, 6),
 (12, 0),
 (13, 7),
 (14, 0),
 (15, 0),
 (16, 7),
 (17, 7),
 (18, 6),
 (19, 2),
 (20, 0),
 (21, 1),
 (22, 7),
 (23, 7),
 (24, 2),
 (25, 0),
 (26, 2),
 (27, 6),
 (28, 6),
 (29, 7),
 (30, 7),
 (31, 7),
 (32, 0),
 (33, 2),
 (34, 6),
 (35, 7),
 (36, 7),
 (37, 1),
 (38, 0),
 (39, 0),
 (40, 7),
 (41, 0),
 (42, 1),
 (43, 7),
 (44, 7),
 (45, 7),
 (46, 7),
 (47, 2),
 (48, 6),
 (49, 6),
 (50, 6),
 (51, 6),
 (52, 2),
 (53, 6),
 (54, 0),
 (55, 7),
 (56, 6),
 (57, 7),
 (58, 6),
 (59, 6),
 (60, 7),
 (61, 0),
 (62, 0),
 (63, 3),
 (64, 6),
 (65, 7),
 (66, 7),
 (67, 6),
 (68, 7),
 (69, 6),
 (70, 6),
 (71, 6),
 (72, 7),
 (73, 1),
 (74, 0),
 (75, 1),
 (76, 0),
 (77, 0),
 (78, 0),
 (79, 0),
 (80, 0),
 (81, 6),
 (82, 1),
 (83, 4),
 (84, 0),
 (85, 7),
 (86, 6),
 (87, 6),
 (88, 7),
 (89, 6),
 (90, 7),
 (91, 2),
 (92, 7),
 (93, 7),
 (94, 7),
 (95, 7),
 (96, 7),
 (97, 6),
 (98, 6),
 (99, 6),
 (100, 7),

In [110]:
cdf.sort('Group_Block').Group_Block.unique()

array([ 1723,  1769,  1770,  1771,  1772,  1773,  1774,  1780,  1783,
        1784,  1795,  1797,  1798,  1817,  1818,  1820,  2006,  2007,
        2008,  2009,  2014,  2015,  2016,  2017,  2023,  2024,  2029,
        2034,  2035,  2036,  2037,  2040,  2369,  2370,  2371,  2372,
        2373,  2374,  2375,  2380,  2385,  2387,  2777,  2778,  2779,
        2780,  2781,  2782,  2783,  2784,  2785,  2786,  2787,  2788,
        2795,  2796,  2797,  2798,  2800,  2802,  2821,  2822,  2823,
        2981,  2985,  2986,  2987,  2988,  2989,  2990,  2991,  2992,
        2993,  2994,  3003,  3004,  3005,  3006,  3007,  3008,  3009,
        3028,  3029,  3031,  3035,  3036,  3038,  3039,  3050,  3183,
        3184,  3187,  3188,  3189,  3190,  3191,  3192,  3193,  3204,
        3205,  3206,  3207,  3210,  3213,  3214,  3218,  3334,  3335,
        3337,  3338,  3339,  3340,  3341,  3342,  3343,  3344,  3345,
        3346,  3352,  3355,  3356,  3357,  3358,  3360,  3361,  3419,
        3568,  3569,